In [1]:
###########################################
# IMPORTS
###########################################
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from simtk import unit, openmm
from perses.tests.utils import compute_potential_components
from openmmtools.constants import kB
from perses.dispersed.utils import configure_platform
from perses.annihilation.rest import RESTTopologyFactory, RESTTopologyFactoryV2
from perses.annihilation.lambda_protocol import RESTStateV2
import numpy as np
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.testsystems import AlanineDipeptideVacuum, AlanineDipeptideExplicit
import itertools

#############################################
# CONSTANTS
#############################################
temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
REFERENCE_PLATFORM = openmm.Platform.getPlatformByName("CUDA")

conducting subsequent work with the following platform: CUDA


INFO:rdkit:Enabling RDKit 2021.03.3 jupyter extensions


In [2]:
def compare_energy_components(rest_system, other_system, positions, platform=REFERENCE_PLATFORM):
    """
    Get energy components of a given system
    """
    platform = configure_platform(platform)

    # Create thermodynamic state and sampler state for non-rest system
    thermostate_other = ThermodynamicState(system=other_system, temperature=temperature)

    # Create context for non-rest system
    integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
    context_other = thermostate_other.create_context(integrator_other)
    context_other.setPositions(positions)

    # Get energy components for non-rest system
    components_other = [component for component in compute_potential_components(context_other, beta=beta)]
    
    # Create thermodynamic state for rest_system
    thermostate_rest = ThermodynamicState(system=rest_system, temperature=temperature)

    # Create context for rest system
    integrator_rest = openmm.VerletIntegrator(1.0 * unit.femtosecond)
    context_rest = thermostate_rest.create_context(integrator_rest)
    context_rest.setPositions(positions)

    # Get energy components for rest system
    components_rest = [component for component in compute_potential_components(context_rest, beta=beta)]

    print(components_other)
    print(components_rest)
    
    # Check that bond, angle, and torsion energies match
    for other, rest in zip(components_other[:3], components_rest[:3]):
        assert np.isclose([other[1]], [rest[1]]), f"The energies do not match for the {other[0]}: {other[1]} (other system) vs. {rest[1]} (REST system)"

    # Check that nonbonded energies match
    nonbonded_other = np.array([component[1] for component in components_other[3:]]).sum()
    nonbonded_rest = np.array([component[1] for component in components_rest[3:]]).sum()
    assert np.isclose([nonbonded_other], [nonbonded_rest]), f"The energies do not match for the NonbondedForce: {nonbonded_other} (other system) vs. {nonbonded_rest} (REST system)"

    print("Energy bookkeeping was a success!")


Alanine dipeptide in vacuum

In [26]:
# Create vanilla system
ala = AlanineDipeptideVacuum()
system = ala.system
positions = ala.positions
topology = ala.topology



In [37]:
# # Create REST system
# system.removeForce(4)
# res1 = list(ala.topology.residues())[1]
# rest_atoms = [atom.index for atom in res1.atoms()]
# factory = RESTTopologyFactory(system, solute_region=rest_atoms)
# REST_system = factory.REST_system



INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
INFO:REST:No unknown forces.


In [4]:
# Create REST system
system.removeForce(4)
res1 = list(ala.topology.residues())[1]
rest_atoms = [atom.index for atom in res1.atoms()]
factory = RESTTopologyFactoryV2(system, topology, rest_region=rest_atoms)
REST_system = factory.REST_system



INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
INFO:REST:No unknown forces.


In [5]:
# Compare energy components
compare_energy_components(REST_system, system, positions)


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.0345517625414407), ('HarmonicAngleForce', 0.607135795359402), ('PeriodicTorsionForce', 3.2298480734947246), ('NonbondedForce', -39.186929599617706), ('AndersenThermostat', 0.0)]
[('CustomBondForce', 0.03455176254144571), ('CustomAngleForce', 0.6071357953594432), ('CustomTorsionForce', 3.2298523942437534), ('NonbondedForce', -39.186929599617905), ('CustomNonbondedForce', 0.0), ('CustomBondForce', 0.0), ('AndersenThermostat', 0.0)]
Energy bookkeeping was a success!


In [3]:
def test_bookkeeping():
    """
    Given the default Global variables, do energy component bookkeeping with the REST system and the original system.
    Specifically, the first CustomBondForce must match the HarmonicBondForce
    (same with the second and third forces with the HarmonicAngle and PeriodicTorsionForces).
    The sum of the NonbondedForce, the CustomNonbondedForce, and the CustomBondForce (last one) must be equal to the
    energy of the original system's NonbondedForce.
    """

    ## CASE 1: alanine dipeptide in vacuum
    # Create vanilla system
    ala = AlanineDipeptideVacuum()
    system = ala.system
    positions = ala.positions
    topology = ala.topology

    # Create REST system
    system.removeForce(4)
    res1 = list(ala.topology.residues())[1]
    rest_atoms = [atom.index for atom in res1.atoms()]
    factory = RESTTopologyFactoryV2(system, topology, rest_region=rest_atoms)
    REST_system = factory.REST_system

    # Compare energy components
    compare_energy_components(REST_system, system, positions)

    ## CASE 2: alanine dipeptide in solvent
    # Create vanilla system
    ala = AlanineDipeptideExplicit()
    system = ala.system
    positions = ala.positions
    topology = ala.topology

    # Create REST system
    system.removeForce(4)
    res1 = list(ala.topology.residues())[1]
    rest_atoms = [atom.index for atom in res1.atoms()]
    factory = RESTTopologyFactoryV2(system, topology, rest_region=rest_atoms, use_dispersion_correction=True)
    REST_system = factory.REST_system

    # Compare energy components
    compare_energy_components(REST_system, system, positions)

    ## CASE 3: alanine dipeptide in solvent with repartitioned hybrid system
    # Create repartitioned hybrid system for lambda 0 endstate
    atp, system_generator = generate_atp(phase='solvent')
    htf = generate_dipeptide_top_pos_sys(atp.topology,
                                         new_res='THR',
                                         system=atp.system,
                                         positions=atp.positions,
                                         system_generator=system_generator,
                                         conduct_htf_prop=True,
                                         repartitioned=True,
                                         endstate=0,
                                         validate_endstate_energy=False)

    # Create REST-ified hybrid system
    res1 = list(htf.hybrid_topology.residues)[1]
    rest_atoms = [atom.index for atom in list(res1.atoms)]
    factory = RESTTopologyFactoryV2(htf.hybrid_system, htf.hybrid_topology, rest_region=rest_atoms, use_dispersion_correction=True)
    REST_system = factory.REST_system

    # Compare energy components
    compare_energy_components(REST_system, htf.hybrid_system, htf.hybrid_positions)


In [4]:
test_bookkeeping()

INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
INFO:REST:No unknown forces.
INFO:REST:Handling constraints
INFO:REST:Handling bonds
INFO:REST:Handling angles
INFO:REST:Handling torsions
INFO:REST:Handling nonbondeds
INFO:REST:Handling nonbonded scaling (custom nb)
INFO:REST:Handling nonbonded exception scaling (custom bond)


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.0345517625414407), ('HarmonicAngleForce', 0.607135795359402), ('PeriodicTorsionForce', 3.2298480734947246), ('NonbondedForce', -39.186929599617706), ('AndersenThermostat', 0.0)]
[('CustomBondForce', 0.03455176254144571), ('CustomAngleForce', 0.6071357953594432), ('CustomTorsionForce', 3.2298523942437534), ('NonbondedForce', -39.186929599617905), ('CustomNonbondedForce', 0.0), ('CustomBondForce', 0.0), ('AndersenThermostat', 0.0)]
Energy bookkeeping was a success!


INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=3.2852863, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=3.2861648000000003, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=3.1855098), unit=nanometer)]
INFO:REST:No unknown forces.
INFO:REST:Handling constraints
INFO:REST:Handling bonds
INFO:REST:Handling angles
INFO:REST:Handling torsions
INFO:REST:Handling nonbondeds
INFO:REST:Handling nonbonded scaling (custom nb)
INFO:REST:Handling nonbonded exception scaling (custom bond)


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03455181083219596), ('HarmonicAngleForce', 0.6071348482604662), ('PeriodicTorsionForce', 3.229847881682588), ('NonbondedForce', -9889.978580573495), ('AndersenThermostat', 0.0)]
[('CustomBondForce', 0.03455181083219596), ('CustomAngleForce', 0.6071348482604662), ('CustomTorsionForce', 3.2298511799221608), ('NonbondedForce', -9889.978580573495), ('CustomNonbondedForce', 0.0), ('CustomBondForce', 0.0), ('AndersenThermostat', 0.0)]
Energy bookkeeping was a success!


DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{9: 7}, {10: 7}, {11: 7}, {12: 7}, {13: 7}, {9: 8}, {10: 8}, {11: 8}, {12: 8}, {13: 8}, {9: 9}, {10: 9}, {11: 9}, {12: 9}, {13: 9}]
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{10: 7}
INFO:proposal_generator:{11: 7}
INFO:proposal_generator:{12: 7}
INFO:proposal_generator:{13: 7}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
IN

...

conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced potential before atom placement: 9.11648114171204


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
added energy components: [('CustomBondForce', 0.000520203927326505), ('CustomAngleForce', 0.4511197756248138), ('CustomTorsionForce', 7.250463328533253), ('CustomBondForce', 12.970883364040223)]


INFO:geometry:total reduced energy added from growth system: 20.672986672125617
INFO:geometry:final reduced energy 29.78946596595156
INFO:geometry:sum of energies: 29.789467813837657
INFO:geometry:magnitude of difference in the energies: 1.8478860965842614e-06
INFO:geometry:Final logp_proposal: -27058.24545987491
INFO:relative:*** Generating RepartitionedHybridTopologyFactory ***
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors add

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.8557755688478691), ('HarmonicAngleForce', 4.379496313777144), ('PeriodicTorsionForce', 34.66577885821952), ('NonbondedForce', -1110.7425025765424), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
[('CustomBondForce', 0.8557755688478691), ('CustomAngleForce', 4.379496313777144), ('CustomTorsionForce', 34.66578505730389), ('NonbondedForce', -1110.7425025765424), ('CustomNonbondedForce', 0.0), ('CustomBondForce', 0.0), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Energy bookkeeping was a success!
